In [3]:
API_KEY = ''
API_SECRET = ''

from binance.client import AsyncClient, Client
client = AsyncClient(API_KEY, API_SECRET)

Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7f9ffc507520>


In [60]:
import pandas as pd
from tqdm import tqdm_notebook as tqdm

In [95]:
historical_klines = pd.DataFrame()
async for kline in await client.get_historical_klines_generator("BTCUSDT", Client.KLINE_INTERVAL_1MINUTE, f"5000 minutes ago UTC"):
    close_price = kline[4]
    close_time = kline[6]
    historical_klines = historical_klines.append({'time': pd.to_datetime(close_time, unit='ms'), 'price': close_price}, ignore_index=True)

In [96]:
historical_klines, future_klines = historical_klines[:2500], historical_klines[2500:]

In [97]:
def get_strategies(df, long_window, short_window):
    sign = None
    strategies = []
    
    df['mean_long'] = df['price'].rolling(long_window).mean()
    df['mean_short'] = df['price'].rolling(short_window).mean()
    
    for row in df.iterrows():
        mean_short, mean_long = row[1]['mean_short'], row[1]['mean_long']
        
        if pd.isna(mean_long):
            strategies.append(pd.NA)
            continue

        local_sign = (mean_short - mean_long) >= 0
        strategy = 0

        if sign == None:
            sign = local_sign

        if local_sign != sign: # знак поменялся
            if local_sign: # поменялся с - на +
                strategy = 1
            else: # поменялся с + на -
                strategy = -1

        sign = local_sign
        strategies.append(strategy)
        
    return strategies

In [98]:
def backtest(df, long_window, short_window):
#     df['mean_long'] = df['price'].rolling(long_window).mean()
# #     set_trace()
#     df['mean_short'] = df['price'].rolling(short_window).mean()
    df['strategy'] = get_strategies(df, long_window, short_window)
    df = df.iloc[(long_window-1):,]
    df['strategy'] = pd.to_numeric(df['strategy'])
    df['price'] = pd.to_numeric(df['price'])
    df.loc[:, 'buy_price'] = df.loc[:, 'price'] * df.loc[:, 'strategy']
    df = df[df['strategy'] != 0]
    df['next_price'] = df['buy_price'].shift(-1)
    df = df[~df['next_price'].isna()]
    df['profit'] = np.where(
        df['strategy'] == -1,
        (df['price'] - df['next_price']) + df['buy_price'] * COMMISION_RATE - df['next_price'] * COMMISION_RATE,
        -(df['next_price'] + df['price']) - df['buy_price'] * COMMISION_RATE + df['next_price'] * COMMISION_RATE)

    return df['profit'].sum()

In [99]:
df = historical_klines.copy()

recount_interval = 15
short_window = 15
long_window = 20

for i, row in tqdm(enumerate(future_klines.iterrows()), total=2500):
    close_price = row[1]['price']
    close_time = row[1]['time']
    df = df.append({'price': close_price, 'time': close_time}, ignore_index=True)
    
    if i % recount_interval == 0:
        results = {}

        for s in range(5, 45, 5):
            for l in range(s + 5, 50, 5):
                results[backtest(df.copy(), l, s)] = (l, s)
                
        long_window, short_window = results[max(results)]
        print("Updated strategy: ", long_window, short_window)
    
    df['strategy'] = get_strategies(df.copy(), long_window, short_window)

  0%|          | 0/2500 [00:00<?, ?it/s]

Updated strategy:  20 15
Updated strategy:  20 15
Updated strategy:  15 10
Updated strategy:  15 10
Updated strategy:  20 15
Updated strategy:  15 10
Updated strategy:  15 10
Updated strategy:  20 15
Updated strategy:  20 15
Updated strategy:  45 35
Updated strategy:  45 35
Updated strategy:  45 35
Updated strategy:  45 35
Updated strategy:  45 35
Updated strategy:  45 35
Updated strategy:  45 35
Updated strategy:  45 35
Updated strategy:  45 35
Updated strategy:  45 35
Updated strategy:  45 35
Updated strategy:  45 35
Updated strategy:  45 35
Updated strategy:  45 35
Updated strategy:  45 35
Updated strategy:  45 35
Updated strategy:  45 35
Updated strategy:  45 35
Updated strategy:  45 35
Updated strategy:  45 35
Updated strategy:  45 35
Updated strategy:  45 35
Updated strategy:  45 35
Updated strategy:  45 35
Updated strategy:  45 35
Updated strategy:  45 35
Updated strategy:  45 35
Updated strategy:  45 35
Updated strategy:  45 35
Updated strategy:  45 35
Updated strategy:  45 35


In [88]:
def calc_profit(df):
#     df['mean_long'] = df['price'].rolling(long_window).mean()
# #     set_trace()
#     df['mean_short'] = df['price'].rolling(short_window).mean()
#     df['strategy'] = get_strategies(df, long_window, short_window)
    df = df[~df['strategy'].isna()]
    df['strategy'] = pd.to_numeric(df['strategy'])
    df['price'] = pd.to_numeric(df['price'])
    df.loc[:, 'buy_price'] = df.loc[:, 'price'] * df.loc[:, 'strategy']
    df = df[df['strategy'] != 0]
    df['next_price'] = df['buy_price'].shift(-1)
    df = df[~df['next_price'].isna()]
    df['profit'] = np.where(
        df['strategy'] == -1,
        (df['price'] - df['next_price']) + df['buy_price'] * COMMISION_RATE - df['next_price'] * COMMISION_RATE,
        -(df['next_price'] + df['price']) - df['buy_price'] * COMMISION_RATE + df['next_price'] * COMMISION_RATE)

    return df['profit'].sum()

In [101]:
(calc_profit(df.copy()) / 47509.36000000) * 100

0.6319686520720835

In [100]:
calc_profit(df.copy())

300.2442620000736

In [93]:
df[df['strategy']!=0]

,time,price,strategy
0,2021-09-15 13:06:59.999,47397.72000000,<NA>
1,2021-09-15 13:07:59.999,47456.31000000,<NA>
2,2021-09-15 13:08:59.999,47439.25000000,<NA>
3,2021-09-15 13:09:59.999,47466.35000000,<NA>
4,2021-09-15 13:10:59.999,47462.35000000,<NA>
5,2021-09-15 13:11:59.999,47477.89000000,<NA>
6,2021-09-15 13:12:59.999,47511.42000000,<NA>
7,2021-09-15 13:13:59.999,47526.84000000,<NA>
8,2021-09-15 13:14:59.999,47511.60000000,<NA>
9,2021-09-15 13:15:59.999,47482.95000000,<NA>


In [40]:
import numpy as np
COMMISION_RATE = 0.0001

In [66]:
import warnings
warnings.filterwarnings('ignore')

In [87]:
df['strategy'].isna()

0       True
1       True
2       True
3       True
4       True
       ...  
395    False
396    False
397    False
398    False
399    False
Name: strategy, Length: 400, dtype: bool